In [1]:
!pip show langchain_community
# Install the langchain_community package
%pip install langchain_community

Name: langchain-community
Version: 0.3.19
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /Users/prathamwankhede/miniconda3/envs/scribe/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
file_path = ("Reading notes.pdf")

In [3]:
loader = PyPDFLoader(file_path=file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)


In [48]:
print(f"{pages[0].metadata}\n")
print(pages[2].page_content)

{'producer': 'macOS Version 15.1 (Build 24B83) Quartz PDFContext', 'creator': 'OneNote', 'creationdate': "D:20250316160924Z00'00'", 'moddate': "D:20250316160924Z00'00'", 'source': 'Reading notes.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}

Nomologically Valid: no counter examples that do not violate the laws of natureConceptually Valid: no counter examples that do not violate the conceptual connections between words
Formally valid: when validity is dependent on Form rather than context of the premisesA is either an X or a Y . A isn’t a Y . ∴A is an X .                           Can we find a counterexample?
Validity:Rules out the possibility of the premises and conclusions not being true at once - It is possible for the conclusion to be false but the argument valid- If the conclusion is false then one of the premises must be false- Not about the actual truth or falsity of sentences, it is about whether it is possible for all premises to be true and conclusion to not be true-


In [11]:
%pip install llama-cloud 

Note: you may need to restart the kernel to use updated packages.


In [14]:
with open('Reading notes.pdf', 'rb') as f:
    file = client.files.upload_file(upload_file=f)

In [ ]:
%pip install llama-cloud-services llama-index-core llama-index-readers-file python-dotenv nest_asyncio

In [ ]:
import nest_asyncio
from dotenv import load_dotenv
nest_asyncio.apply()
load_dotenv()

from llama_cloud_services import LlamaParse
from llama_index.core import SimpleDirectoryReader


parser = LlamaParse(
    result_type = "markdown",
    extract_layout=True
)

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=['Reading notes.pdf'], file_extractor=file_extractor).load_data()
documents

In [ ]:
from IPython.display import Markdown,display
display(Markdown(documents[6].text))

In [49]:
%pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 2.4 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [50]:
%pip install pillow PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 4.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import base64
import io

import fitz
from PIL import Image

def pdf_to_base_64(pdf_path: str, page_number: int):
    pdf_doocument = fitz.open(pdf_path)
    page = pdf_doocument.load_page(page_number)
    image = page.get_pixmap()
    image = Image.frombytes("RGB", [image.width, image.height], image.samples)
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

In [41]:
import base64
from PIL import Image
import io

def combine_base64_images_vertically(base64_images):
    """
    Combines multiple base64-encoded images into one vertically stacked image.

    Args:
        base64_images (list): List of base64-encoded image strings.

    Returns:
        str: Base64-encoded string of the combined image.
    """
    images = [Image.open(io.BytesIO(base64.b64decode(img))) for img in base64_images]

    # Calculate the total height and maximum width of the combined image
    total_height = sum(img.height for img in images)
    max_width = max(img.width for img in images)

    # Create a blank image with the calculated dimensions
    combined_image = Image.new("RGB", (max_width, total_height))

    # Paste each image into the combined image
    y_offset = 0
    for img in images:
        combined_image.paste(img, (0, y_offset))
        y_offset += img.height

    # Convert the combined image back to base64
    buffer = io.BytesIO()
    combined_image.save(buffer, format="JPEG")
    combined_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return combined_base64

In [ ]:
from IPython.display import Image as IPImage
from IPython.display import display

base64_image = pdf_to_base_64(file_path, 0)
display(IPImage(data=base64.b64decode(base64_image)))

In [42]:
imagearray = []

for pagenum in range(pages[0].metadata["total_pages"]):
    base64_image = pdf_to_base_64(file_path, pagenum)
    imagearray.append(base64_image)

In [ ]:
from IPython.display import Image as IPImage
from IPython.display import display

output = combine_base64_images_vertically(imagearray)
display(IPImage(data=base64.b64decode(output)))

In [ ]:
from openai import OpenAI
import os
import dotenv

dotenv.load_dotenv()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

completion = client.chat.completions.create(
    model="google/gemma-3-27b-it:free",
    messages=[
        {
            "role": "user", 
            "content": [
                {
                    "type": "text",
                    "text": "Read the following text and return only the text form of this in markdown while preserving the formatting and tables."
                },
                {
                    "type": "image_url",
                    "image_url":{ 
                        "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                }
            ]
         }
    ]
)

In [103]:
display(Markdown(completion.choices[0].message.content))

```markdown
DS, m, n

Modus Tollens
```
m 
¬→
¬

MT, m, n

Double negation Elimination:
```
m 
¬¬→
¬
```

DNE, m

Excluded Middle:
```
j 
→
→
```

AS

```
j 
¬→
¬
```

AS

LEM, l-j, k-l

Soundness and Completeness:
Concept | Truth table (semantic) definition | Proof-theoretic (syntactic) definition
------- | -------- | --------
Tautological theorem | A sentence whose truth table has Ts under the main connective | A sentence that can be derived without any premises
Contradiction | A sentence whose truth table only has Fs under the main connective | A sentence whose negation can be derived without any premises
Contingent sentence | A sentence whose truth table contains both Ts and Fs under the main connective | A sentence that is not a theorem nor a contradiction
Equivalent sentences | The columns under the main connective are identical | The sentences can be derived from each other
Equatisfiable sentences | Sentences which do not have a single line in their truth table where they are all true |  Sentences from which one can derive the contradiction 
Satisfiable Consistent sentences | Sentences which have at least one line in their truth table where they are all true | Sentences from which one cannot derive the contradiction 
Valid argument | An argument where all its truth table lines where there are Ts under all its premises for the premise and Fs under the main connective for the conclusion | An argument where one can derive the conclusion from the premises

Logical property | To prove it present | To prove it absent
----------------- | ------------------- | -------------------
Being a theorem | Derive the sentence | Find a false line in the truth table for the sentence
Being a contradiction | Derive the negation of the sentence | Find a true line in the truth table for the sentence
Contingency | Find a false and a true line in the truth table of the sentence | Prove the sentence and its negation
Equivalence | Derive each sentence from the other | Find a line in the truth table for the sentence where they have different values
Consistency | Find a line in truth table for the sentence where they all are true | Derive a contradiction from the sentences
Validity | Derive the conclusion from the premises | Find a line in the truth table where the premises are true and the conclusion false.
```

In [ ]:
from openai import OpenAI
import os
import dotenv

dotenv.load_dotenv()

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY")
)

completion = client.chat.completions.create(
    model="llama-3.2-90b-vision-preview",
    messages=[
        {
            "role": "user", 
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI that extracts text from an image of a PDF page and converts it into Markdown while preserving its structure. Follow these steps: "+
                    "1. Carefully extract all readable text from the image."+
                    "2. Preserve headers, lists, bold, italicized text, and any structured formatting."+
                    "3. Convert the text into Markdown format without adding external information."+
                    "4. Ensure accurate indentation and organization of content."+
                    "After completing these steps, return only the final form of the Markdown text. No need for anything else apart from the final text. Do not use any external resources or information."
                },
                {
                    "type": "image_url",
                    "image_url":{ 
                        "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                }
            ]
         }
    ]
)

In [ ]:
display(Markdown(completion.choices[0].message.content))

In [6]:
file_path = ("Reading notes.pdf")
loader = PyPDFLoader(file_path=file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)


In [7]:
def imgtomarkdown(img):
    completion = client.chat.completions.create(
        model="llama-3.2-90b-vision-preview",
        messages=[
                {
                    "role": "user", 
                    "content": [
                        {
                            "type": "text",
                            "text": "Take the following image and return the text form of only what is in the image in markdown while preserving the formatting and tables. Do not any extra information."
                        },
                        {
                            "type": "image_url",
                            "image_url":{ 
                                "url": f"data:image/jpeg;base64,{img}"
                                }
                        }
                    ]
                }
            ]
    )

    return completion.choices[0].message.content



In [5]:
from openai import OpenAI
import os
import dotenv

def imgToMDRouter(img):
    dotenv.load_dotenv()

    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=os.getenv("OPENROUTER_API_KEY")
    )

    completion = client.chat.completions.create(
        model="google/gemma-3-27b-it:free",
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": "You are an AI that extracts text from an image of a PDF page and converts it into Markdown while preserving its structure. Follow these steps: "+
                    "1. Carefully extract all readable text from the image."+
                    "2. Preserve headers, lists, bold, italicized text, and any structured formatting."+
                    "3. Convert the text into Markdown format without adding external information."+
                    "4. Ensure accurate indentation and organization of content."+
                    "5. Make sure to not put any tables in code blocks for optimal parsing"+
                    "Make sure the final Markdown text is properly formatted and strcutured. This must be parseable by a Markdown parser. Do not put it in a code block."+
                    "Do not use code blocks at all."
                    },
                    {
                        "type": "image_url",
                        "image_url":{ 
                            "url": f"data:image/jpeg;base64,{img}"
                            }
                    }
                ]
            }
        ]
    )
    return completion.choices[0].message.content

In [ ]:
def imgToMDRouter(img):
    dotenv.load_dotenv()

    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=os.getenv("OPENROUTER_API_KEY")
    )

    completion = client.chat.completions.create(
        model="google/gemma-3-27b-it:free",
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": """
                            You are an AI that extracts text from an image of a PDF page and converts it into Markdown while preserving its structure. Pay close attention to lists, headings, and logical formatting. Do *not* introduce information not present in the original text.  Do *not* include code blocks.

Here are some examples of how to convert PDF text to Markdown, paying attention to structure.  Includes examples of both regular text *and* truth tables:

**Example 1 (Standard Text):**

**Input (PDF Text):**
Validity:
Rules out the possibility of the premises and conclusions not being true at once -
It is possible for the conclusion to be false but the argument valid-

**Output (Markdown):**
**Validity:**
Rules out the possibility of the premises and conclusions not being true at once -
It is possible for the conclusion to be false but the argument valid-

**Example 2 (List):**
**Input (PDF Text):**
Practice Questions:
A:
Valid1.Valid2.
Invalid3.Valid4.
Invalid5.Invalid6.

**Output (Markdown):**
Practice Questions:

A:
1. Valid
2. Valid
3. Invalid
4. Valid
5. Invalid
6. Invalid

**Example 3 (Truth Table):**

**Input (PDF Text):**
Column Index(i)3210
2iT & F8 T & F4 T & F2 T & F1 T & F

**Output (Markdown):**

| Column Index (i) | 3 | 2 | 1 | 0 |
|---|---|---|---|---|
| 2i | T | & | F | 8 |
| T | & | F | 4 | T |
| & | F | 2 | T | F |
| 1 | T | & | F |  |

**Example 4 (Combined Text & Logical Notation):**
**Input (PDF Text):**
A If A, then C 
∴C

**Output (Markdown):**
A If A, then C
∴C

Now, convert the following PDF into Markdown, adhering to the examples above. Pay special attention to accurately converting any truth tables that might be present.

                            """
                    },
                    {
                        "type": "image_url",
                        "image_url":{ 
                            "url": f"data:image/jpeg;base64,{img}"
                            }
                    }
                ]
            }
        ]
    )
    return completion.choices[0].message.content

In [6]:
import dotenv
import os

totalpages = pages[0].metadata["total_pages"]
totalmarkdown = ""

for pagenum in range(totalpages):
    img = pdf_to_base_64(file_path, pagenum)
    totalmarkdown += imgToMDRouter(img)
        

In [7]:
from IPython.display import Markdown,display    

display(Markdown(totalmarkdown))

# Reading notes

Wednesday, September 11, 2019 11:44 PM

Nomologically Valid: no counter examples that do not violate the laws of nature
Conceptually Valid: no counter examples that do not violate the conceptual connections between words.

Formally valid: when validity is dependent on form rather than context of the premises
*   A is either an X or a Y.
*   A isn’t a Y.
*   :. A is an X.

Can we find a counterexample?

Validity:
*   Rules out the possibility of the premises and conclusions not being true at once
*   It is possible for the conclusion to be false but the argument valid.
*   If the conclusion is false then one of the premises must be false.
*   Not about the actual truth or falsity of sentences, it is about whether it is possible for all premises to be true and conclusion to not be true.

Soundness:
*   Argument is sound if it is both valid and all its premises are true. 

Inductive arguments: generalize observations about past events to a conclusion about future cases
*   It is still possible for that conclusion to be refuted with a counter example

Entailment:
*   The relationship between premises and conclusion

Practice Questions:

1.  Valid
2.  Invalid
3.  Valid
4.  Invalid
5.  Valid
6.  Invalid
B. True

Joint possibility: Sentences that are not possible to be true at once
*   If the sentences are joint impossible, you cannot add another sentence to fix it
*   If the premises are join impossible -> the argument is valid
Joint Possible: all sentences must be possible to exist at the same time

Necessary Truths, Necessary Falsehoods and contingences:
*   Contingent: a sentence that is capable of being true and capable of being false
*   Truth: Something that is just true in general
*   Falsehood: A sentence that is impossible

Equivalence: when two sentences have the same truth-value in every case
*   These sentences are typically similar to each other, if one is false then the other also has to be false (vice versa)
John went to the store after he washed the dishes.
John washed the dishes before he went to the store.

A
If A, then C
¬C 
¬A

Conceptually Valid: the concepts carry through
Arguments are not valid based *solely* on their form

TFL: Truth Functional logic

Atomic Sentences:
*   Add subscripts to letters to represent subsentences and use a symbolization key such as
    *   A₁: It is raining outside
    *   C₁: Jenny is miserable

Logical Connectives:

| symbol | what it is called | rough meaning |
|---|---|---|
| ¬ | negation | “It is not the case that...”|
| ∧ | conjunction | “Both … and …”|∨ conjunction “...and...” or ...
→ conditional “If ... then...”
↔ biconditional “...if and only if...”

Negation
* “It is not the case that” or “not” – you are negating the sentence
* It is possible to have a double negation given the conditions
* Jane is unhappy
* Jane is not unhappy
Sentence B is its own sentence because it could be that Jane is neither happy or unhappy - and it is not the same thing as “It is not the case that Jane is happy”

Conjunction
* We use it to deal with “and”, it is used to connect two subsentences together. The subsentences are the conjuncts of the conjunction.
* It does not matter if you have multiple conjunctions
* Even if you are referring to one subject, you still must define the two subsentences
* A sentence can be symbolized as (A & B) if it can be paraphrased in English as “Both... and...” or “as ... and...” or as “although”
* You can ignore “but”

Disjunction
* Subsentences are the disjuncts of the disjunction
* A sentence can be symbolized as (A ∨ B) if it can be paraphrased in English as “Either...or...”
* Inclusive or. Allows for the possibility of both
* Exclusive or. Excludes the possibility of both

Conditional
* If, then “→” – use “→” to symbolize “if... then...” format when P and Fare subsentences
* P is the Antecedent of the conditional
* Fis the Consequent of the conditions
* Be careful as the sentence may be different if the subsentences are switched
* A sentence can be symbolized as (A → B) if it can be paraphrased in English as “If A, then B” or “only if”
* Can have many forms – look out for this
* Tells us that only if the antecedent is true, then the consequent is true
* Does not discuss a casual connection between two events

Biconditional
* “...if and only if...”
* ∨ – the English biconditional
* A sentence can be symbolized as (A ↔ B) if it can be paraphrased in English as “A if B”, that is, as “if

Unless: A special case
¿ You will wear a jacket
¿ You will catch a cold
* Unless you wear a jacket, you will catch a cold. (~J → C)
* You will catch a cold unless you wear a jacket. (J → C)
* All [J ∨ C] – Both sentences are equal in their meanings as such this symbolization works too, it is not possible for these two have an equal in the same time
* At Three of these are considered to be equal

Expression of TFL: Any string of symbols of TFL
Sentences of TFL: An Inductive definition
* Every Sentence letter is a sentence
* If A is a sentence then –A is also a sentence
* If A and B are sentences, then (A connectivity B) is a sentence
* Nothing else is a sentence.

Main Logical Operation:
* a sentenceal connective that is introduced last
* We can find it by:
- If the first symbol in the sentence is – then it is the main logical operator
- Otherwise, count brackets. For each open bracket, add 1, for each closing one, subtract 1, the counter is 0 exactly 1, the first operator you hit (apart from –) is the main logical operator

Scope of Negation: The scope of “–“ is the subsentence for which “–“ is the main logical operator

Scope of a connective: the subsentence for which that connective is the main logical operator

Bracket Conventions:
* Allowed to omit the outermost bracket of the sentence
* In more complex sentences, we can use “...” for faster reading

Ambiguity
Logical Ambiguity: when a sentence contains a word(s) that can have multiple meanings.
Structural Ambiguity: a sentence can be interpreted in different ways.Vagueness: when something is not specified yet rather described, (i.e. “all” but in what context?)

Sentences in TFL cannot be structurally ambiguous.

Scope ambiguities: depend on whether or not a connective is in the scope of another.

Quotation Conventions:
*   Indicate that you are moving from talking about an object to talk about a name of that object

Object Language and metalanguage:
*   Object Language: TFL
*   Metalanguage: English
*   Difference between these two is in their exactness

Metavariables:
*   Augmented metalanguage that is used to talk about any expression of TFL
*   i.e. cursive A is used to describe the expressions of TFL while A is a specific sentence letter of TFL

Quotation conventions (for arguments):
*   TFL cannot identify cert
*   ain sentences as premises and conclusions
*   Notation: 

    ⟨A₁, …, Aₙ⟩ ⊢ ⟨ℭ⟩

“therefore” is not part of object language, is part of metalanguage

Truth Tables:
*   Negation:

    ⟨¬⟩
    T F
    F T

*   Conjunction: If A and B are then True

    ⟨∧⟩
    T T T
    T F F
    F T F
    F F F

*   Disjunction: If A or B is true then True (inclusive or)

    ⟨∨⟩
    T T T
    T F T
    F T T
    F F F

*   Conditional: If A is true then B is True

    ⟨→⟩
    T T T
    T F F
    F T T
    F F T

*   Biconditional: Both must be true for the expression to evaluate to true

    ⟨↔⟩
    T T T
    T F F
    F T F
    F F T

The idea of Truth Functionality:
*   A connective is Truth-functional if the truth value of a sentence with that connective as its main logical operator is uniquely determined by the truth value(s) of the constituent sentence(s)
*   Every connective in TFL is truth-functional
*   To determine the truth value of a TFL sentence, we only need to know the truth value of its components.
*   “is necessarily the case” is not truth-functional

Symbolizing vs. Translating:
*   All connectives of TFL map us between truth valuesTFL ignores the subtleties of the original text which enhance it beyond just truth values
It also cannot capture the subtle differences between sentences.
TFL cannot work with meaning as well.

When we treat a TFL sentence as symbolizing an English sentence, we are stipulating that the TFL sentence is
to take the same truth value as that English sentence.

Indicative vs. Subjunctive conditionals
- The truth value of the sentence is not uniquely determined by the truth value of the parts
- Subjunctive conditionals ask us to imagine something contrary to fact, and then ask us to evaluate what
would have happened in that case

Complete truth tables:
- We can assign truth values directly: A valuation
- Each row of the truth table represents a possible valuation; the complete table represents all the possible
valuations - it allows us to calculate the truth value of complex sentences
- The values under the main logical operator tell us the value of the sentence

H   | (H∧T) →H
----|-----------
T   |  T T T T
T   |  T F F T
F   |  F F F T
F   |  F F F T

- A complete truth table has a line for every possible assignment of True and False to the relevant sentence
letters. Each line is a valuation and the truth table has a line for all the different valuations
- The size of said truth table can vary based on the number of sentence letters in the table.
- A sentence that only needs 1 sentence letter requires only two rows [or T and F].
- This is true even if the same letter is repeated multiple times

C   | (C ⊃ C) ⟷ (C ∧ (¬C ⊃ C))
----|---------------------------
T   |  T T T T T
T   |  T T T F F
F   |  F F F F T
F   |  F F F F T

- We can continue this trend infinitely based on the number of sentence letters we have.
- For example, with 3 letters:

M   | N   | P   | M ∧ (N ∨ P)
----|-----|-----|-------------
T   | T   | T   |  T T T T
T   | T   | F   |  T T F T
T   | F   | T   |  T F T T
T   | F   | F   |  T F F F
F   | T   | T   |  F T T T
F   | T   | F   |  F T F F
F   | F   | T   |  F F T F
F   | F   | F   |  F F F F

- A rule for this is that if we have n different sentence letters, then the table must have 2ⁿ rows/lines
- To fill the columns, start from right to left
 1. At the right most, alternate between T and F.
 2. In the next column to its left, write two T’s and two F’s and alternate/repeat
 3. In the next left column, write 4 T’s and 4 F’s and alternate/repeat
 4. Continue this trend…
RULE:
Column Index(s) | 3| 2 | 1 | 0
---------------|---|---|---|---
2<sup>T & F</sup> | 8 | 4 | 2 | 1

brackets:
- If the same connective is used (conjunction or disjunction) throughout a sentence, then associative property
applies.

Example: Logically equivalent sentences
((A ∨ B) ∨ C)
(A ∨ (B ∨ C))

- We should not write: A ∨ B ∨ C
- Associative property does not apply to conditionals, or if there is variation of the conjunctions and
disjunctions in the sentence, then the order of the brackets is important and there is no logical equivalence

Example of conditionals:
(A → B) → C
((A → B) → C)

Example of Variation:
((A ∨ B) A C)
(A ∨ (B A C))

- It is then needless to say that we cannot write these without their brackets. It would be far to ambiguous.

Unknown Concepts:the surrogate for a necessary truth is a surrogate is a true or every valuation. It is to first symbolize in TFL, an example of this “2+2 =4".
There are some necessary truths that we cannot adequately symbolize in TFL, an example of this is no sentence letter can be tautology by itself.

As such, the opposite to this is a contradiction. A is a contradiction if it is false on every valuation.
Another variation of this is equivalence: A and B are equivalent if for every valuation, their truth values are the same.

An example of this:

P Q ~(P∨Q) ~P∧~Q
T T F T T F F T
T F F T T F F F
F T F F T T F F
F F T F F T T T

“If A~, then A” is a substitution in TFL. The surrogate for entailment: A₁, A₂, … Aₙ C
This is not a symbol of TFL rather it is a symbol of metalanguage.
P·P →Q and P↔Q is the same thing as the TFL sentences “P” and “P ↔entail “Q”

Some special cases of the tumstile are as follows:

- Since there are no sentences on the left side of the tumstile, this simply means that there is no valuation that makes C false, as such ever valuation makes C true.
Thus C is a tautology

A~
- Similarly, for this case we can say that A is a contradiction

Furthermore, to say that it is not the case that A₁, … Aₙ C, we can say A₁, A₂, … Aₙ ~C
This means that some valuation makes all of A₁, … Aₙ true whilst making C false, this is not the same is A₁, … Aₙ~C for it is possible that some other valuation makes A₁, … Aₙ true and makes C true.
As such: P ⊨ Q but also P~⊨Q

“versus”
A ⊨ C if no valuation of the sentence letters makes A true and C false
A ⊨ C is a tautology if no valuation of the sentence letters makes A true
A ⊨ C is a tautology if A ⊨ C
"~" is a sentential connective of TFL
"⊨" is a symbol of augmented English.
When we use the tumstile we are forming a metalinguistic sentence that mentions the surrounding TFL sentences

Limitations of TFL

TFL is not expressive enough to symbolize everything we might want to so that we can use the skills we have.
Sentence letters cannot be allowed to take truth values other than True or false, it cannot capture the intended truth conditions.
Some logical conventions in the English language cannot be properly captured by the TFL connectives, namely the material conditional, these are called the paradoxes of the material conditional.

In any idea of natural selection:

- The tables cannot give much insight in certain cases where different forms of reasoning are present.
- The aim of a natural deduction system is to show that particular arguments are valid in a way that we understand the reasoning the arguments implied.

Complicated arguments can become untraceable in TFL. If we choose to use truth tables, there must be a more efficient way to

Conditional:

```
a  |  a -> b
---+---------
b  |
```

To get b, you must first satisfy a. When citing a reason you must write the line of the conditional first and then the antecedent after that. We can further make an additional assumption for the sake of the argument to show that we are no longer dealing with just our original premise.

We are not claiming, we are assuming. We must mark this on our sheet.

```
i   |  a
j   |  a -> b
----+---------
    |  b      ->I, i-j
```

Example:
1   P -> Q   PR
2   Q -> R   PR
3   P          AS
4   Q          ->E, 1, 3
5   R          ->E, 2, 4
6   P -> R    ->I, 3-5

Additional Assumption and Subproofs:
Once a subproof is closed, we cannot use anything within that proof.
To cite an individual line when applying a rule:
1. the line must come before the line where the rule is applied, but
2. not occur within a subproof that has been closed before the line where the rule is applied.
By closing a subproof, we are discharging the assumptions of the subproof, we cannot use anything that was derived using the discharged assumption.
Any assumption works to start a subproof with

```
i   |  a
j   |  a -> b   AS
----+---------
    |  a       AS
    |  b       ->I, i-j
```

Biconditional:
Must prove both A and B

```
m   |  a <-> b
----+---------
n   |  a      AS
o   |  b      AS
    |  a <-> b   ->I,m, n
```

Disjunction:

```
m   |  a v b
----+---------
j   |  a       AS
k   |  b       AS
    |  a v b   ->E, j, k
```

Negation:

```
m   |  a
----+---------
n   |  -a
    |  -a        ->E, m, n
```

Disjunctive Syllogism:

```
m   |  a v b
----+---------
n   |  -a
    |  b       ->E, m, n
```DS, m, n

Modus Tollens
m 
→
n
¬n
MT, m, n

Double negation Elimination:
m
¬¬m 
→
d
DNE, m

Excluded Middle:
f
→
AS
¬f
→
AS
LEM, l, j, k-l

Soundness and Completeness:
Concept 
Truth table (semantic) definition
A sentence whose truth table only has T under the main connective. 
Proof-theoretic (syntactic) definition
A sentence that can be derived without any premises. 

Contradiction/
Impossibility
A sentence whose truth table only has F under the main connective.
A sentence whose negation can be derived without any premises. 

Contingent sentence
A sentence which has both a true line and a false line under the main connective. 
A sentence that is not a theorem or contradiction. 

Equivalant/
Interdefinition
The columns under the main connective are identical.
The sentences can be derived from each other. 

Satisfiable/
Consistent sentence 
Sentences which have at least one line in their truth table where they are all true.
Sentences from which one can derive the contradiction &

Valid argument
An argument where all truth table lines where there are all T’s under all premises for the premises and F for the conclusion.
An argument where one can derive the conclusion from the premises. 

Logical property
To prove it present
To prove it absent
Being a theorem
Derive the sentence
Find a false line in the truth table for the sentence.

Being a contradiction 
Derive the negation of the sentence
Find a true line in the truth table for the sentence.

Contingency
Find a false line and a true line in the truth table of the sentence
Prove the sentence or its negation. 

Equivalence
Derive each sentence from the other
Find a line in the truth tables for the sentence where they have different values 

Consistency
Find a line in a truth table for the sentence where they all are true
Derive a contradiction from the sentences 

Validity 
Derive the conclusion from the premises
Find a line in the truth table where the premises are true and the conclusion false.

In [8]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers=[
    ("\n", "New Line"),
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("**", "Bold"),
    ("*", "Italic"),
    ("1.", "Numbered List"),
    ("```", "Code Block"),
    ("|", "Table")
    
    
]

MDsplitter = MarkdownHeaderTextSplitter(headers, strip_headers=False)
split = MDsplitter.split_text(totalmarkdown)

In [15]:
split[0].metadata.items()

dict_items([('Header 1', 'Reading notes')])

In [21]:
%pip install -qU langchain-huggingface

In [32]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [ ]:
for i in range(len(split)):
    split[i] = split[i].replace("\n", " ")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm

embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

/Users/prathamwankhede/miniconda3/envs/scribe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
import hashlib
from tqdm.auto import tqdm

documentHash = hashlib.sha512(b"Reading\ notes.pdf").hexdigest()

vsdb = InMemoryVectorStore(embedding=embedding_model)
docs = []
ids = []
for i in tqdm(range(len(split))):
    docs.append(Document(page_content=split[i].page_content,
                        metadata={"document": documentHash[:7]}))
    ids.append(hashlib.sha512(split[i].page_content.encode()).hexdigest()[:7])

vsdb.add_documents(documents=docs, ids=ids)

  0%|          | 0/83 [00:00<?, ?it/s]

100%|██████████| 83/83 [00:00<00:00, 6106.53it/s]


['0bf90ad',
 'c2d32b4',
 '81b8f52',
 'ff01735',
 'be8c778',
 '06133fe',
 '3db6252',
 'e7745e2',
 '3fcaa3f',
 'ef68250',
 '744a1dd',
 '5281f6b',
 'ade9dc6',
 '3cc343a',
 '33cca63',
 '22b32b8',
 'c6fba14',
 '630b77c',
 '5358839',
 '62cc305',
 'bf8f42c',
 'f985de0',
 '9fe6889',
 '9ac85f3',
 'ad36107',
 'b16f1c3',
 '485fab5',
 '18b099f',
 'b1eaabf',
 'acb6e2d',
 'a9537f0',
 '0d12379',
 'af6b9bf',
 '08df70b',
 'a8fb0b3',
 '0f7f5e5',
 '64f60a9',
 '93902fc',
 'b7fc4b9',
 '560328d',
 '09e269d',
 '7ccb8da',
 '7ff3a38',
 'a92bcce',
 '8076dfe',
 '6483ffd',
 'e555372',
 '076c17d',
 '3bd5430',
 '6e94957',
 'aa7db04',
 '02a1816',
 'dba1b1d',
 '136dcb4',
 'c7387df',
 '29e1561',
 'e1de4cd',
 '1e45e76',
 '940c2ab',
 '1d84b1f',
 '654c1bd',
 '770dcc4',
 '81f4ddb',
 'd48a9d5',
 '1d3ddbd',
 '41ef66a',
 '58ff6a8',
 '7d7f422',
 'a7d3d39',
 'd82bc00',
 'cfe3115',
 'b27e105',
 'ae08cea',
 'b5feb77',
 '6b06ce0',
 '3576c12',
 'bffd141',
 'cef073d',
 '7c6e1f9',
 '3b9bb9b',
 'e9fb1e2',
 'ef9d664',
 'c7a7db0']

In [37]:
results = vsdb.similarity_search_by_vector(embedding=embedding_model.embed_query("What are the arguments against TFL?"))

In [38]:
results

[Document(id='1d3ddbd', metadata={'document': '1954152'}, page_content='*   Augmented metalanguage that is used to talk about any expression of TFL'),
 Document(id='770dcc4', metadata={'document': '1954152'}, page_content='*   Object Language: TFL'),
 Document(id='41ef66a', metadata={'document': '1954152'}, page_content='*   i.e. cursive A is used to describe the expressions of TFL while A is a specific sentence letter of TFL  \nQuotation conventions (for arguments):'),
 Document(id='bffd141', metadata={'document': '1954152'}, page_content='*   To determine the truth value of a TFL sentence, we only need to know the truth value of its components.')]

In [29]:
results[3].page_content

'*   TFL cannot identify cert'

In [25]:
split

[Document(metadata={'Header 1': 'Reading notes'}, page_content='# Reading notes  \nWednesday, September 11, 2019 11:44 PM  \nNomologically Valid: no counter examples that do not violate the laws of nature\nConceptually Valid: no counter examples that do not violate the conceptual connections between words.  \nFormally valid: when validity is dependent on form rather than context of the premises'),
 Document(metadata={'Italic': 'A is either an X or a Y.'}, page_content='*   A is either an X or a Y.'),
 Document(metadata={'Italic': 'A isn’t a Y.'}, page_content='*   A isn’t a Y.'),
 Document(metadata={'Italic': ':. A is an X.'}, page_content='*   :. A is an X.  \nCan we find a counterexample?  \nValidity:'),
 Document(metadata={'Italic': 'Rules out the possibility of the premises and conclusions not being true at once'}, page_content='*   Rules out the possibility of the premises and conclusions not being true at once'),
 Document(metadata={'Italic': 'It is possible for the conclusion to